In [1]:
! pip install pyonmttok -q
! pip install OpenNMT-py -q

     |████████████████████████████████| 16.9 MB 546 kB/s 
     |████████████████████████████████| 226 kB 2.2 MB/s 
     |████████████████████████████████| 116 kB 11.3 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 73 kB 1.4 MB/s 
     |████████████████████████████████| 1.3 MB 28.3 MB/s 


In [2]:
#Sentencepiece Install - pip install doesn't give access to cmd tools.

! sudo apt-get install cmake build-essential pkg-config libgoogle-perftools-dev
! git clone https://github.com/google/sentencepiece.git 
%cd sentencepiece
!mkdir build
%cd build
!cmake ..
!make -j $(nproc)
!sudo make install
!sudo ldconfig -v

%cd /content

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libunwind-dev
The following NEW packages will be installed:
  libgoogle-perftools-dev libunwind-dev
0 upgraded, 2 newly installed, 0 to remove and 12 not upgraded.
Need to get 627 kB of archives.
After this operation, 6,761 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libunwind-dev amd64 1.2.1-8 [423 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgoogle-perftools-dev amd64 2.5-2.2ubuntu3 [204 kB]
Fetched 627 kB in 2s (324 kB/s)
debconf: unable to ini

In [ ]:
!spm_encode -h

In [8]:
!wget https://github.com/sawradip/Bang2Eng/blob/main/bn.model?raw=true -O bn.model 
!wget https://github.com/sawradip/Bang2Eng/blob/main/en.model?raw=true -O en.model 

--2022-10-09 16:01:00--  https://github.com/sawradip/Bang2Eng/blob/main/bn.model?raw=true
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/sawradip/Bang2Eng/raw/main/bn.model [following]
--2022-10-09 16:01:00--  https://github.com/sawradip/Bang2Eng/raw/main/bn.model
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sawradip/Bang2Eng/main/bn.model [following]
--2022-10-09 16:01:00--  https://raw.githubusercontent.com/sawradip/Bang2Eng/main/bn.model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1196728 (1.1M) [applicati

In [9]:
# Download base_bn2en.pt
!gdown 1xx5bU31sIMU24kLm5bYh19qLBYHHriP4

Downloading...
From: https://drive.google.com/uc?id=1xx5bU31sIMU24kLm5bYh19qLBYHHriP4
To: /content/base_bn2en.pt
100% 331M/331M [00:03<00:00, 94.3MB/s]


In [10]:
import os
import shutil
import torch

In [11]:
! echo "সেরা আন্তর্জাতিক সিনেমা বিভাগে ছবিটি মনোনীত হয়েছে।" | spm_encode --model="/content/bn.model"

▁সেরা ▁আন্তর্জাতিক ▁সিনেমা ▁বিভাগে ▁ছবিটি ▁মনোনীত ▁হয়েছে ।


In [24]:

def spmModel2Vocab(model_path):
    model_prefix = model_path.split(os.path.sep)[-1].split('.')[0]
    temp_dir = os.path.sep.join(model_path.split(os.path.sep)[:-1])
    vocab_cmd = [
        "spm_export_vocab --model",
        model_path,
        "| tail -n +4 >",
        model_path.replace('.model', '.vocab')
    ]
    os.system(" ".join(vocab_cmd))
    print(f"{ model_path.replace('.model', '.vocab') } generated.")


def spmOperate(CFG, temp_dir, tokenize):
    if tokenize:
        modelName = os.path.join(temp_dir, f"srcSPM.model")
        input_file = CFG.input_txt

        spm_cmd = [
            f"spm_encode --model=\"{modelName}\"",
            f"--output_format=piece",
            f"< \"{input_file}\" > \"{os.path.join(temp_dir, 'srctxt')}.tok\""
        ]

        os.system(" ".join(spm_cmd))
        print('Tokenization Completed')
        return f"{os.path.join(temp_dir, 'srctxt')}.tok"
           

    else:
        tgtTok_path = f"{os.path.join(temp_dir, 'tgttxt')}.tok"
        output_file = os.path.join(CFG.output_dir, CFG.output_filename)

        modelName = os.path.join(temp_dir, f"tgtSPM.model")
        spm_cmd = [
            f"spm_decode --model=\"{modelName}\"",
            f"< \"{tgtTok_path}\" > \"{output_file}\""
        ]
        os.system(" ".join(spm_cmd))
        # os.remove(input_file)
        post_cmd = f"""sed 's/▁/ /g;s/  */ /g' -i \"{output_file}\""""
        os.system(post_cmd)
        return output_file
  

def translate(CFG, srcTok_path):
    tgtTok_path = srcTok_path.replace('srctxt', 'tgttxt')
    cmd = f'''
        onmt_translate \
            -model \"{CFG.src2tgt_model}\" \
            -src \"{srcTok_path}\" \
            -output \"{tgtTok_path}\" \
            -replace_unk -verbose -max_length {CFG.tgt_seq_length} -batch_size {CFG.eval_batch_size} {'-gpu 0' if torch.cuda.is_available() else ''}
    '''
    print('Translation Starting...')
    os.system(cmd)

    print('Translation Completed. Detokenizing...')
    return tgtTok_path



In [36]:

def bang2eng_translate(bn_sentence = None, bn_textfile = None):
    class CFG:
        src2tgt_model = './base_bn2en.pt'
        src_lang_model = './bn.model'
        tgt_lang_model = './en.model'
        output_dir = '.'

    if (bn_sentence and bn_textfile) or not (bn_sentence or bn_textfile):
        raise Exception("Provide one of bn_sentence or bn_textfile")

    temp_dir = os.path.join(CFG.output_dir, 'temp')

    if os.path.isdir(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)

    if bn_sentence:
        bn_textfile = os.path.join(temp_dir, 'bn_textfile.txt')
        with open( bn_textfile, 'w') as writer:
            writer.write(bn_sentence)
    print(bn_textfile)
    CFG.input_txt = bn_textfile
    CFG.output_filename = os.path.join('temp', 'preds.txt') if bn_sentence else 'preds.txt'
    CFG.tgt_seq_length = 1024
    CFG.eval_batch_size = 8
    
    srcModel_tempPath = os.path.join(temp_dir, "srcSPM.model")
    shutil.copy(CFG.src_lang_model, srcModel_tempPath)

    tgtModel_tempPath = os.path.join(temp_dir, "tgtSPM.model")
    shutil.copy(CFG.tgt_lang_model, tgtModel_tempPath)

    srcVocab_tempPath = spmModel2Vocab(srcModel_tempPath)
    tgtVocab_tempPath = spmModel2Vocab(tgtModel_tempPath)

    srcTok_tempPath = spmOperate(CFG, temp_dir, tokenize = True)

    tgtTok_tempPath = translate(CFG, srcTok_tempPath)

    en_textfile = spmOperate(CFG, temp_dir, tokenize = False)

    with open(en_textfile) as reader:
        en_sentence = reader.read()
    
    return en_sentence



### Single sentence

In [37]:
t = bang2eng_translate(bn_sentence = 'সেরা আন্তর্জাতিক সিনেমা বিভাগে ছবিটি মনোনীত হয়েছে।')


./temp/bn_textfile.txt
./temp/srcSPM.vocab generated.
./temp/tgtSPM.vocab generated.
Tokenization Completed
Translation Starting...
Translation Completed. Detokenizing...


In [ ]:
print(t)

### Sentence Batch (text file)

In [42]:

with open('/content/RisingNews.txt', 'w') as writer:
            writer.write(
"""বিমানবন্দরে যুক্তরাজ্যে নিযুক্ত বাংলাদেশ হাইকমিশনার সাঈদা মুনা তাসনীম ও লন্ডনে বাংলাদেশ মিশনের জ্যেষ্ঠ কর্মকর্তারা তাকে বিদায় জানান।
সন্ত্রাসীদের কোনো বর্ণ, ধর্ম ও দেশ নেই।
কর্নেল বিশ্ববিদ্যালয়ের অধ্যাপক ডেরেক কাবেরা "শ্রেণিবদ্ধ চিন্তার অন্ধকার দিক" নামের একটি প্রবন্ধ রচনা করেছেন।
জনসাধারণ প্রতি কেজি পেঁয়াজ ৪৫ টাকায়, চিনি ৫০ টাকায়, মশুর ডাল ৫০ টাকায় ও প্রতি লিটার সয়াবিন তেল ৮৫ টাকায় কিনতে পারবে বলে জানান তিনি।
"""
)

t = bang2eng_translate(bn_textfile = '/content/RisingNews.txt')


/content/RisingNews.txt
./temp/srcSPM.vocab generated.
./temp/tgtSPM.vocab generated.
Tokenization Completed
Translation Starting...
Translation Completed. Detokenizing...


In [39]:
print(t)

Syeda Muna Tasnim, the Bangladesh High Commissioner to the United Kingdom and senior officials of Bangladesh Mission in London, said good-bye to her.
The terrorists have no caste, religion and country.
Professor Derek Kabera of Cornell University wrote an article entitled "The Dark Side of Classified Thought".
He said that the people could buy onions at Tk 45 per kg, sugar Tk 50, lentils at Tk 50 and soybean oil at Tk 85 per liter.

